# 준비
## 라이브러리 호출

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## 데이터 셋 로드

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train.shape, x_test.shape

11493376/11490434 [==============================] - 1s 0us/step


((60000, 28, 28), (10000, 28, 28))

## 데이터셋 전처리

채널 추가

In [3]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

데이터 셋 셔플, batch 지정

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 모델

모델 서브클래싱(subclassing) API

In [5]:
class MyModel(Model):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


# Create an instance of the model
model = MyModel()

옵티마이저, 손실함수, 지표 설정

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 훈련

In [7]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Accuracy: {train_accuracy.result() * 100}, '
          f'Test Loss: {test_loss.result()}, '
          f'Test Accuracy: {test_accuracy.result() * 100}')

Epoch 1, Loss: 0.13661609590053558, Accuracy: 95.86833190917969, Test Loss: 0.06485461443662643, Test Accuracy: 97.81999969482422
Epoch 2, Loss: 0.04190641641616821, Accuracy: 98.68000030517578, Test Loss: 0.05403048172593117, Test Accuracy: 98.18999481201172
Epoch 3, Loss: 0.021948738023638725, Accuracy: 99.3066635131836, Test Loss: 0.04725433513522148, Test Accuracy: 98.29999542236328
Epoch 4, Loss: 0.013814999721944332, Accuracy: 99.53500366210938, Test Loss: 0.07900860905647278, Test Accuracy: 97.73999786376953
Epoch 5, Loss: 0.01016742642968893, Accuracy: 99.65333557128906, Test Loss: 0.05780957266688347, Test Accuracy: 98.5199966430664
